# FXの将来のリターン動向の予測モデル

目的：過去18時間の **"usdjpy","cadjpy","audjpy","eurjpy","zarjpy","sgdjpy","nzdjpy","gbpjpy","chfjpy"** を使って、将来**1時間後**のリターン動向を予測する   
モデル：GCN   
開発環境: python 3.11.5/ JupyterLab 3.6.3/Jupyter Notebook Version: 6.5.4/System: Linux #14~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from torch_geometric.nn.pool import global_mean_pool

import numpy as np
import pandas as pd

from tslearn.metrics import dtw
from sklearn.metrics import precision_score, recall_score, roc_auc_score, average_precision_score, f1_score, accuracy_score
from tqdm.auto import tqdm

Pandas version: 2.0.3   
Numpy version: 1.24.3   
sklearn: 1.3.0   
sklearn: 0.6.3   
torch: 2.1.1   
torch_geometric: 2.4.0   
tqdm: 4.65.0

In [33]:
# Load datasets
'''
9 currency pairs from 2018-01-01 18:10:00 to 2023-11-17 16:10:00
'''

def load_dataset(filename, start_date, end_date):
    
    df = pd.read_csv(f'processed_data/{filename}.csv',sep = ',')
    df.set_index('Times', inplace=True)
    
    df = df.loc[start_date:end_date]
    df.reset_index(inplace=True)
    
    return df

start_date = '2021-11-17 18:10:00'
end_date = '2023-11-17 16:10:00'

variable_names = ["usdjpy","cadjpy","audjpy","eurjpy","zarjpy","sgdjpy","nzdjpy","gbpjpy","chfjpy"]
for name in variable_names:
    exec(f"{name} = load_dataset(str.upper('{name}'), start_date, end_date)")
    exec(f"display({name}.head())")

,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency
0,2021-11-17 18:10:00,114.210,0.256397,0.001437,1,0
1,2021-11-17 18:11:00,114.216,0.256515,0.001297,1,0
2,2021-11-17 18:12:00,114.218,0.256555,0.001183,1,0
3,2021-11-17 18:13:00,114.227,0.256732,0.000946,1,0
4,2021-11-17 18:14:00,114.226,0.256713,0.000972,1,0


,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency
0,2021-11-17 18:10:00,90.565,0.447782,0.001835,1,1
1,2021-11-17 18:11:00,90.566,0.447809,0.001558,1,1
2,2021-11-17 18:12:00,90.569,0.447889,0.001757,1,1
3,2021-11-17 18:13:00,90.575,0.448050,0.001503,1,1
4,2021-11-17 18:14:00,90.571,0.447943,0.001458,1,1


,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency
0,2021-11-17 18:10:00,82.974,0.595548,0.001399,1,2
1,2021-11-17 18:11:00,82.984,0.595807,0.001242,1,2
2,2021-11-17 18:12:00,82.980,0.595703,0.000494,1,2
3,2021-11-17 18:13:00,82.982,0.595755,0.000687,1,2
4,2021-11-17 18:14:00,82.976,0.595600,0.000699,1,2


,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency
0,2021-11-17 18:10:00,129.252,0.297177,0.000573,1,3
1,2021-11-17 18:11:00,129.256,0.297258,0.000929,1,3
2,2021-11-17 18:12:00,129.259,0.297318,0.000805,1,3
3,2021-11-17 18:13:00,129.271,0.297559,0.000650,1,3
4,2021-11-17 18:14:00,129.268,0.297498,0.000681,1,3


,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency
0,2021-11-17 18:10:00,7.342,0.477536,-0.001722,0,4
1,2021-11-17 18:11:00,7.357,0.481571,0.000469,1,4
2,2021-11-17 18:12:00,7.345,0.478343,-0.001014,0,4
3,2021-11-17 18:13:00,7.344,0.478074,-0.001000,0,4
4,2021-11-17 18:14:00,7.341,0.477267,-0.001258,0,4


,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency
0,2021-11-17 18:10:00,84.241,0.283314,0.002222,1,5
1,2021-11-17 18:11:00,84.248,0.283493,0.001616,1,5
2,2021-11-17 18:12:00,84.252,0.283596,0.001568,1,5
3,2021-11-17 18:13:00,84.258,0.283750,0.001639,1,5
4,2021-11-17 18:14:00,84.257,0.283724,0.001651,1,5


,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency
0,2021-11-17 18:10:00,79.954,0.643968,0.002392,1,6
1,2021-11-17 18:11:00,79.961,0.644190,0.002191,1,6
2,2021-11-17 18:12:00,79.959,0.644127,0.002279,1,6
3,2021-11-17 18:13:00,79.946,0.643715,0.001364,1,6
4,2021-11-17 18:14:00,79.954,0.643968,0.001727,1,6


,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency
0,2021-11-17 18:10:00,154.114,0.463693,0.002326,1,7
1,2021-11-17 18:11:00,154.119,0.463771,0.002209,1,7
2,2021-11-17 18:12:00,154.105,0.463554,0.001838,1,7
3,2021-11-17 18:13:00,154.110,0.463631,0.001533,1,7
4,2021-11-17 18:14:00,154.101,0.463492,0.001286,1,7


,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency
0,2021-11-17 18:10:00,122.992,0.260529,0.001660,1,8
1,2021-11-17 18:11:00,123.000,0.260654,0.001807,1,8
2,2021-11-17 18:12:00,123.000,0.260654,0.001790,1,8
3,2021-11-17 18:13:00,123.004,0.260716,0.001407,1,8
4,2021-11-17 18:14:00,123.003,0.260700,0.001464,1,8


In [34]:
# Combine all the datasets
combined_df = pd.concat([usdjpy, cadjpy, audjpy, eurjpy, zarjpy, sgdjpy, nzdjpy, gbpjpy, chfjpy], axis=1)
display(combined_df.head())

,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency,Times,Final Price,Final Price Normalized,1h_Return,...,Final Price Normalized,1h_Return,Label,Currency,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency
0,2021-11-17 18:10:00,114.210,0.256397,0.001437,1,0,2021-11-17 18:10:00,90.565,0.447782,0.001835,...,0.463693,0.002326,1,7,2021-11-17 18:10:00,122.992,0.260529,0.001660,1,8
1,2021-11-17 18:11:00,114.216,0.256515,0.001297,1,0,2021-11-17 18:11:00,90.566,0.447809,0.001558,...,0.463771,0.002209,1,7,2021-11-17 18:11:00,123.000,0.260654,0.001807,1,8
2,2021-11-17 18:12:00,114.218,0.256555,0.001183,1,0,2021-11-17 18:12:00,90.569,0.447889,0.001757,...,0.463554,0.001838,1,7,2021-11-17 18:12:00,123.000,0.260654,0.001790,1,8
3,2021-11-17 18:13:00,114.227,0.256732,0.000946,1,0,2021-11-17 18:13:00,90.575,0.448050,0.001503,...,0.463631,0.001533,1,7,2021-11-17 18:13:00,123.004,0.260716,0.001407,1,8
4,2021-11-17 18:14:00,114.226,0.256713,0.000972,1,0,2021-11-17 18:14:00,90.571,0.447943,0.001458,...,0.463492,0.001286,1,7,2021-11-17 18:14:00,123.003,0.260700,0.001464,1,8


In [35]:
# Function to compute technical indicator
'''
Calculates each technical indicator using the entire hour for each node.
'''

# Calculate Bollinger Bands for the entire hour
def calculate_bollinger_bands(data):
    window = len(data)
    rolling_mean = data.rolling(window=window).mean()
    upper_band = rolling_mean + 2 * data.rolling(window=window).std()
    lower_band = rolling_mean - 2 * data.rolling(window=window).std()
    
    return upper_band.iloc[-1], lower_band.iloc[-1], rolling_mean.iloc[-1]

# Calculate RSI for the entire hour
def calculate_rsi(data):
    diff = data.diff(1).dropna()
    gain = diff.where(diff > 0, 0)
    loss = -diff.where(diff < 0, 0)

    avg_gain = gain.mean()
    avg_loss = loss.mean()

    if avg_loss == 0:
        rs = np.inf  # Set to infinity to avoid division by zero
    else:
        rs = avg_gain / avg_loss

    rsi = 100 - (100 / (1 + rs))
    return rsi

# Calculate RCI for the entire hour
def calculate_rci(data):
    rci = data.pct_change().sum()
    return rci

# Calculate Momentum for the entire hour
def calculate_momentum(data, n=59):
    return data.diff(n).iloc[-1]

In [36]:
# Function to create a single graph (snapshot)

'''
In each graph we use data within 18 hours.
Each hour each currency pair forms a node, so each snapshot contains 18*9　nodes.
Weighted edges are generated from computing DTW of two nodes (two 1 hour series) using "Final Price Normalized"
Node features consist of "Final Price Normalized", "1h_Return", "Currency"　and technical indicator
'''

def create_graph(snapshot, node_window=60, node_stride=60):
    
    features = snapshot[[ 'Final Price Normalized','1h_Return', 'Currency', 'Final Price']].values
    
    series = []
    currencies = []
    prices = []
    h_returns = []
    Final_prices = []
    volatilities = []
    
    upper_bands = []
    lower_bands = []
    rolling_means = []
    rsis = []
    rcis = []
    momenta = []  
    
     
    for i in range(0, len(snapshot) - node_window + 1, node_stride):
        for j in range (9): 
            serie = features[i:i+node_window,j]
            series.append(serie) 
            

            currency = features[i+node_window-1,j+18]
            currencies.append(currency)

            price = features[i+node_window-1, j]
            prices.append(price)
            
            Final_price = features[i:i+node_window, j+27]
            Final_prices.append(Final_price)

            h_return = features[i+node_window-1, j+9]
            h_returns.append(h_return)
            
            
            # Create a DataFrame with the stock price data
            df = pd.DataFrame({'Close': Final_price})

            # Calculate Bollinger Bands
            upper_band, lower_band, mean = calculate_bollinger_bands(df['Close'])
            upper_bands.append(upper_band)
            lower_bands.append(lower_band)
            rolling_means.append(mean)

            # Calculate RSI for the entire hour using the function
            rsi = calculate_rsi(df['Close'])
            rsis.append(rsi)

            # Calculate RCI for the entire hour using the function
            rci = calculate_rci(df['Close'])
            rcis.append(rci)
            
            # Calculate Momentum for the entire hour using the function
            momentum = calculate_momentum(df['Close'])
            momenta.append(momentum)
            
            all_return = features[i:i+node_window, j+9]
            
            # Create a DataFrame with the returns
            df_r = pd.DataFrame({'Return': all_return})
            # Compute volatility (standard deviation of returns)
            volatility = df_r['Return'].std()
            volatilities.append(volatility)
            
            
    # Edge generation 
    adjacency_matrix = np.zeros((len(series), len(series)))
    
    for i in range(len(series)):
        for j in range(i+1, len(series)):
            dtw_distance = dtw(series[i], series[j])

            # Update the maximum observed DTW distance dynamically
            max_dtw_distance = max(max_dtw_distance, dtw_distance) if 'max_dtw_distance' in locals() else dtw_distance
    
    for i in range(len(series)):
        for j in range(i+1, len(series)):
            dtw_distance = dtw(series[i], series[j])
            similarity = 1 - (dtw_distance / max_dtw_distance)

            
            adjacency_matrix[i, j] = similarity
            adjacency_matrix[j, i] = similarity
            
    np.fill_diagonal(adjacency_matrix, 1)
    # print(adjacency_matrix)

    feature_matrix = np.transpose(np.vstack((prices, h_returns, currencies, upper_bands, lower_bands, rolling_means, rsis, rcis, momenta, volatilities)))

    return adjacency_matrix, feature_matrix

In [54]:
# Function to create graphs
'''
Build a graph using 18 hours of data, then move forward by 1 hour and build the next graph using the next 18 hours of data,
store them in a list graphs.
Time window of each graph is 18*60, stride is 60.

The label of each graph is the 'Lable' of the next hour's data, ("Label" is 1 if "1h_return" is larger than 1, 0 otherwise)
Because we want to use each graph(18 hours data) to predict the next hour data.
The label of each graph is a sequence of 9 binary values, each represents a currency.
'''

def create_graphs(data, time_window=18*60, stride=60):
    adjacency_matrice = []
    feature_matrice = []
    graphs = []
    labels = []
    
    # Get label data
    Label = data['Label'].values
    
    # Iterate through data with a sliding window
    for i in tqdm(range(0, len(data) - 2*time_window + 1, stride)):
    # Get a snapshot of the time series data
        snapshot = data.iloc[i:i + time_window]
        
        # Generate adjacency matrix and feature matrix from the snapshot
        adjacency_matrix, feature_matrix = create_graph(snapshot)
        
        # Get the label
        label = labels_data[i + time_window + 61]
        
        # Create edge index indicating positions of non-zero elements
        edge_index = torch.tensor([[i, j] for i in range(adjacency_matrix.shape[0]) for j in range(adjacency_matrix.shape[1]) if adjacency_matrix[i, j] != 0], dtype=torch.long).t().contiguous()
        
        # Convert to PyTorch Tensors
        x_tensor = torch.FloatTensor(feature_matrix)
        y_tensor = torch.LongTensor(label)       

        # Create a PyTorch Geometric Data object
        graph = Data(x=x_tensor, edge_index=edge_index)
        
        # Append to the lists
        graphs.append(graph)
        labels.append(y_tensor)

    return graphs, labels

In [55]:
#GCN
'''
The task is graph classification.
Output is the label of each graph which is a sequence of 9 binary values, each represents a currency.
'''

class GCNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.linear1 = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr, data.batch

        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = self.conv2(x, edge_index, edge_weight)

        # Readout layer, take feature-wise average values of all node embeddings and use it as a graph feature of an input graph (readout)
        x = global_mean_pool(x, batch)
        
        # Transform a graph feature by a linear transformation
        x= self.linear1(x)
        
        #Employ a sigmoid function as an activation function of the output layer
        x = torch.sigmoid(x)
        return x

In [ ]:
# Generate all snapshots. 
graphs, labels = create_graphs(combined_df)


  0%|          | 0/17483 [00:00<?, ?it/s]

In [75]:
graphs = GRAPHS
labels = LABEL
print(len(graphs))

17483


In [76]:
# Split data into training and testing sets
split_ratio = 0.85
split_idx = int(split_ratio * len(graphs))

train_data = graphs[:split_idx]
train_labels = labels[:split_idx]
test_data = graphs[split_idx:]
test_labels = labels[split_idx:]

In [77]:
# Set batch size for training
batch_size = 32

# Learning rate for optimization
lr = 1.65e-05

# Create DataLoader for training and test data with specified batch size and shuffle
train_loader = DataLoader(list(zip(train_data, train_labels)), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(list(zip(test_data, test_labels)), batch_size=batch_size, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCNModel(input_dim=10, hidden_dim=128, output_dim=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss()

In [ ]:
# Training loop 
num_epochs = 900
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        graphs, labels = batch
        graphs = graphs.to(device)
        labels = labels.to(device)
        labels = labels.float()
        
        optimizer.zero_grad()
        output = model(graphs)
        output = output.float()
        
        # print(output.shape)
        # print(labels.shape)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    if (epoch + 1) % 50 == 0:
        print(f'Epoch:{epoch + 1} \t loss: {loss:.6f}')
        torch.save(model.state_dict(), 'GCN_2pairs_ti.dat')


Epoch:50 	 loss: 0.638808
Epoch:100 	 loss: 0.662785
Epoch:150 	 loss: 0.889476
Epoch:200 	 loss: 0.569334
Epoch:250 	 loss: 0.626111
Epoch:300 	 loss: 0.719108
Epoch:350 	 loss: 0.622538
Epoch:400 	 loss: 0.683809
Epoch:450 	 loss: 0.551279
Epoch:500 	 loss: 0.673702
Epoch:550 	 loss: 0.664974
Epoch:600 	 loss: 0.744123


In [79]:
all_predictions = []
all_labels = []

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation during evaluation
with torch.no_grad():
    for batch in tqdm(test_loader, desc='Evaluating', leave=False):
        graphs, labels = batch
        graphs = graphs.to(device)
        labels = labels.to(device)
        labels = labels.float()

        # Forward pass
        output = model(graphs)
        output = output.float()

        # Convert probabilities and labels to numpy arrays for scikit-learn metrics
        output_np = output.cpu().numpy()
        labels_np = labels.cpu().numpy()

        all_predictions.append(output_np)
        all_labels.append(labels_np)

# Concatenate predictions and labels across batches
all_predictions = np.concatenate(all_predictions).flatten()
all_labels = np.concatenate(all_labels).flatten()

# Binary classification thresholding (you can adjust the threshold if needed)
threshold = 0.5
binary_predictions = (all_predictions > threshold).astype(int)

# # # # Calculate evaluation metrics
precision = precision_score(all_labels, binary_predictions, average='macro')
recall = recall_score(all_labels, binary_predictions, average='macro')
roc_auc = roc_auc_score(all_labels, all_predictions)
aupr = average_precision_score(all_labels, all_predictions)
# f1 = f1_score(all_labels, binary_predictions, average='macro')
accuracy = accuracy_score(all_labels, binary_predictions)
f1 = f1_score(all_labels, binary_predictions, average='macro')

# Print or use the metrics as needed
print(f'Accuracy: {accuracy:.4f}')

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'AUC: {roc_auc:.4f}')
print(f'AUPR: {aupr:.4f}')
print(f'F1 Score: {f1:.4f}') 

Evaluating:   0%|          | 0/82 [00:00<?, ?it/s]

Accuracy: 0.6024
Precision: 0.6599
Recall: 0.6098
AUC: 0.6325
AUPR: 0.6192
F1 Score: 0.5719
